In [1]:
# loading GWSNR class from the gwsnr package
from gwsnr import GWSNR
import numpy as np

# initializing the GWSNR class with default parameters
gwsnr = GWSNR(snr_type='interpolation_aligned_spins_jax')

# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
snrs = gwsnr.snr(mass_1=np.array([30.0]), mass_2=np.array([30.0]), luminosity_distance=np.array([1000.0]), a_1=np.array([0.0]), a_2=np.array([0.0]))

# print the signal-to-noise ratio
print('Computed SNRs:\n', snrs)

GWSNR: Setting multiprocessing start method to 'fork'.
If you need to use the 'spawn' method (in case error or warning due to other library dependencies),
set the environment variable GWSNR_USE_SPAWN=True *before* running your script.
Command line (single line):
    GWSNR_USE_SPAWN=True python yourscript.py
In a Python script (before importing GWSNR):
    import os
    os.environ['GWSNR_USE_SPAWN'] = 'True'
    import gwsnr


Initializing GWSNR class...

psds not given. Choosing bilby's default psds
[{'mtot_min': 9.96, 'mtot_max': 235.0, 'mtot_resolution': 200, 'ratio_min': 0.1, 'ratio_max': 1.0, 'ratio_resolution': 20, 'sampling_frequency': 2048.0, 'waveform_approximant': 'IMRPhenomD', 'minimum_frequency': 20.0, 'detector': 'L1', 'psds': "PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')", 'detector_tensor': '[[ 0.41128086  0.14021027  0.24729459]\n [ 0.

In [7]:
'gwsnr.psds_list[0].psd_array' == 'gwsnr.psds_list[0].psd_array'

True

In [2]:
from gwsnr.utils import load_pickle

test = load_pickle('/Users/phurailatpamhemantakumar/phd/mypackages/gwsnr/examples/interpolator_pickle/L1/param_dict_list.pickle')

In [3]:
test

[{'mtot_min': 9.96,
  'mtot_max': 235.0,
  'mtot_resolution': 200,
  'ratio_min': 0.1,
  'ratio_max': 1.0,
  'ratio_resolution': 20,
  'sampling_frequency': 2048.0,
  'waveform_approximant': 'IMRPhenomD',
  'minimum_frequency': 20.0,
  'detector': 'L1',
  'psds': "PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')",
  'detector_tensor': '[[ 0.41128086  0.14021027  0.24729459]\n [ 0.14021027 -0.10900569 -0.18161563]\n [ 0.24729459 -0.18161563 -0.30227517]]',
  'spin_max': 0.99,
  'spin_resolution': 10},
 {'mtot_min': 9.96,
  'mtot_max': 235.0,
  'mtot_resolution': 200,
  'ratio_min': 0.1,
  'ratio_max': 1.0,
  'ratio_resolution': 20,
  'sampling_frequency': 2048.0,
  'waveform_approximant': 'IMRPhenomXPHM',
  'minimum_frequency': 20.0,
  'detector': 'L1',
  'psds': "PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/en

In [ ]:
PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/gwsnr/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt') 
PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')

In [6]:
import os

asd_file = 'noise/aLIGO_O4_high_asd.txt'
filename = os.path.basename(asd_file)
print(filename)  # Output: aLIGO_O4_high_asd.txt


aLIGO_O4_high_asd.txt


In [2]:
# gerneral case, random parameters
# chirp mass can go upto only 95 if f_min=20. to get non zero SNR
nsamples = 10000
chirp_mass = np.linspace(5,60,nsamples)
mass_ratio = np.random.uniform(0.2,1,size=nsamples)
param_dict = dict(
    mass_1 = (chirp_mass*(1+mass_ratio)**(1/5))/mass_ratio**(3/5),
    mass_2 = chirp_mass*mass_ratio**(2/5)*(1+mass_ratio)**(1/5),
    # Fix luminosity distance
    luminosity_distance = 500*np.ones(nsamples),
    # Randomly sample everything else:
    theta_jn = np.random.uniform(0,2*np.pi, size=nsamples),
    ra = np.random.uniform(0,2*np.pi, size=nsamples), 
    dec = np.random.uniform(-np.pi/2,np.pi/2, size=nsamples), 
    psi = np.random.uniform(0,2*np.pi, size=nsamples),
    phase = np.random.uniform(0,2*np.pi, size=nsamples),
    geocent_time = 1246527224.169434*np.ones(nsamples),
    a_1 = np.random.uniform(-0.8,0.8, size=nsamples),
    a_2 = np.random.uniform(-0.8,0.8, size=nsamples),
)

In [ ]:
## aligned spins
# numba njit: time=28.8 ms ± 339 μs; jax jit: time= 13.1 ms ± 180 μs (forkserver), time= 13.1 ms ± 141 μs (fork)

%timeit snrs = gwsnr.snr_with_interpolation(**param_dict)

13.1 ms ± 141 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
snrs = gwsnr.snr_with_interpolation(**param_dict)

In [ ]:
from gwsnr.utils import save_pickle, load_pickle
